In [1]:
from detectron2.layers import ShapeSpec
from pixel_decoder.msdeformattn import MSDeformAttnPixelDecoder
from transformer_decoder.mask2former_transformer_decoder import MultiScaleMaskedTransformerDecoder
import torch
from detectron2.config import get_cfg
from detectron2.modeling import build_backbone

/home/zbyang/miniconda3/envs/cuda11.7/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Modify ckp weight names due to changes in Detectron2
import pickle
with open("../models/mask2former_R50_COCO_panoptic.pkl", "rb") as f:
    m2f_weights = pickle.load(f, encoding="latin1")

In [30]:
bb_weights = m2f_weights['model']
bb_weights_new = bb_weights.copy()
for k, v in bb_weights.items():
    if k[:12] == 'backbone.res':
        bb_weights_new["backbone.stages." + k[9:]] = v
        bb_weights_new.pop(k)
    elif "pixel_decoder.pixel_decoder" in k:
        new_k = k.replace("pixel_decoder.pixel_decoder", "pixel_decoder")
        print(new_k)
        bb_weights_new[new_k] = v
        bb_weights_new.pop(k)

In [ ]:
bb_weights_new.keys()

In [32]:
m2f_weights['model'] = bb_weights_new
with open("../models/mask2former_R50_COCO_panoptic_new.pkl", "wb") as f:
    pickle.dump(m2f_weights, f)

In [2]:
# Modify ckp weight names due to changes in Detectron2
import pickle
with open("../models/mask2former_R50_COCO_panoptic_new.pkl", "rb") as f:
    m2f_weights_new = pickle.load(f, encoding="latin1")

In [3]:
mtd = torch.load('../pretrained_models/coco/M2F_R50_MSDeformAttnPixelDecoder.pkl', map_location='cpu')

In [9]:
mtd['transformer.encoder.layers.5.self_attn.sampling_offsets.weight'].sum()

tensor(-17.9100)

In [8]:
m2f_weights_new['model']['sem_seg_head.pixel_decoder.transformer.encoder.layers.5.self_attn.sampling_offsets.weight'].sum()

-17.910046

In [3]:
# Load the transformer decoder of Mask2Former
args = {
    "in_channels": 256,
    "mask_classification": True,
    "num_classes": 133,
    "hidden_dim": 256,
    "num_queries": 100,
    "nheads": 8,
    "dim_feedforward": 2048,
    "dec_layers": 9,
    "pre_norm": False,
    "mask_dim": 256,
    "enforce_input_project": False,
}

mtd = MultiScaleMaskedTransformerDecoder(**args)
mtd_weights = torch.load('../pretrained_models/coco/maskformer2_R50_transformer_decoder.pkl')
mtd.load_state_dict(mtd_weights)

<All keys matched successfully>

In [3]:
# Load deformable pixel decoder
input_shape = {"res2": ShapeSpec(channels=256, stride=4),
               "res3": ShapeSpec(channels=512, stride=8),
               "res4": ShapeSpec(channels=1024, stride=16),
               "res5": ShapeSpec(channels=2048, stride=32)}
args = {
    'input_shape': input_shape,
    'conv_dim': 256, 
    'mask_dim': 256, 
    'norm': 'GN',
    'transformer_dropout': 0.0, 
    'transformer_nheads': 8, 
    'transformer_dim_feedforward': 1024, 
    'transformer_enc_layers': 6, 
    'transformer_in_features': ['res3', 'res4', 'res5'], 
    'common_stride': 4,
}

msd = MSDeformAttnPixelDecoder(**args)

msd_weights = torch.load('../pretrained_models/coco/maskformer2_R50_MSDeformAttnPixelDecoder.pkl')
msd.load_state_dict(msd_weights)

RuntimeError: Error(s) in loading state_dict for MSDeformAttnPixelDecoder:
	Missing key(s) in state_dict: "lateral_convs.adapter_1.weight", "lateral_convs.adapter_1.norm.weight", "lateral_convs.adapter_1.norm.bias", "output_convs.layer_1.weight", "output_convs.layer_1.norm.weight", "output_convs.layer_1.norm.bias". 
	Unexpected key(s) in state_dict: "adapter_1.weight", "adapter_1.norm.weight", "adapter_1.norm.bias", "layer_1.weight", "layer_1.norm.weight", "layer_1.norm.bias". 

In [72]:
cfg.MODEL.WEIGHTS

'/sensei-fs/users/zhiboy/projects/Mask2Former/pretrained_models/coco/R50.pkl'

In [71]:
import os
os.path.dirname(cfg.MODEL.WEIGHTS)

'/sensei-fs/users/zhiboy/projects/Mask2Former/pretrained_models/coco'

In [52]:
# Load Detectrion2 Resnet50
cfg = get_cfg()
cfg.merge_from_file('../configs/resnet50.yaml')
backbone = build_backbone(cfg)
r50_weights = torch.load(cfg.MODEL.WEIGHTS)
backbone.load_state_dict(r50_weights)

Config '../configs/resnet50.yaml' has no VERSION. Assuming it to be compatible with latest v2.


<All keys matched successfully>

In [43]:
inp = torch.randn(4, 3, 320, 512)

In [53]:
features = backbone(inp)

In [54]:
a, b, c = msd.forward_features(features)

In [60]:
a.shape, b.shape, [s.shape for s in c]

(torch.Size([4, 256, 80, 128]),
 torch.Size([4, 256, 10, 16]),
 [torch.Size([4, 256, 10, 16]),
  torch.Size([4, 256, 20, 32]),
  torch.Size([4, 256, 40, 64])])

In [62]:
for k, v in features.items():
    print(k, v.shape)

res2 torch.Size([4, 256, 80, 128])
res3 torch.Size([4, 512, 40, 64])
res4 torch.Size([4, 1024, 20, 32])
res5 torch.Size([4, 2048, 10, 16])
